In [1]:
# Agenda
# 1. Finish up the india_export example
# 2. Take a look at the soccer example


In [2]:
# Step 1 is to re-enter the helper functions
import pandas as pd
import sqlite3
from sqlite3 import Error

def create_connection(db_file, delete_db=False):
    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn


def create_table(conn, create_table_sql):
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
def execute_sql_statement(sql_statement, conn):
    cur = conn.cursor()
    cur.execute(sql_statement)

    rows = cur.fetchall()

    return rows

In [3]:
# connect to the "flat" version of the database
conn = create_connection('india_export.db')


In [4]:
# connect to the normalized database
conn_norm = create_connection('india_export_norm.db')

In [5]:
# take a look at the flat database
sql_statement = "SELECT * FROM Export"
df = pd.read_sql_query(sql_statement, conn)
display(df)

,HSCode,Commodity,value,country,year
0,2,MEAT AND EDIBLE MEAT OFFAL.,0.18,AFGHANISTAN TIS,2018
1,3,"FISH AND CRUSTACEANS, MOLLUSCS AND OTHER AQUAT...",0.00,AFGHANISTAN TIS,2018
2,4,DAIRY PRODUCE; BIRDS' EGGS; NATURAL HONEY; EDI...,12.48,AFGHANISTAN TIS,2018
3,6,LIVE TREES AND OTHER PLANTS; BULBS; ROOTS AND ...,0.00,AFGHANISTAN TIS,2018
4,7,EDIBLE VEGETABLES AND CERTAIN ROOTS AND TUBERS.,1.89,AFGHANISTAN TIS,2018
...,...,...,...,...,...
133365,42,"ARTICLES OF LEATHER,SADDLERY AND HARNESS;TRAVE...",0.00,SAMOA,2010
133366,48,"PAPER AND PAPERBOARD; ARTICLES OF PAPER PULP, ...",0.00,SAMOA,2010
133367,49,"PRINTED BOOKDS, NEWSPAPERS, PICTURES AND OTHER...",NaN,SAMOA,2010
133368,52,COTTON.,0.00,SAMOA,2010


In [6]:
# take a look at the normalized database
sql_statement = "SELECT * FROM Year"
df = pd.read_sql_query(sql_statement, conn_norm)
display(df)

,YEAR
0,2010
1,2011
2,2012
3,2013
4,2014
5,2015
6,2016
7,2017
8,2018


In [7]:
sql_statement = "SELECT * FROM Country"
df = pd.read_sql_query(sql_statement, conn_norm)
display(df)

,Country
0,AFGHANISTAN TIS
1,ALBANIA
2,ALGERIA
3,AMERI SAMOA
4,ANDORRA
...,...
243,VIRGIN IS US
244,WALLIS F IS
245,YEMEN REPUBLC
246,ZAMBIA


In [8]:
sql_statement = "SELECT * FROM Commodity"
df = pd.read_sql_query(sql_statement, conn_norm)
display(df)

,HSCODE,COMMODITY
0,1,"AIRCRAFT, SPACECRAFT, AND PARTS THEREOF."
1,2,ALBUMINOIDAL SUBSTANCES; MODIFIED STARCHES; GL...
2,3,ALUMINIUM AND ARTICLES THEREOF.
3,4,ANIMAL OR VEGETABLE FATS AND OILS AND THEIR CL...
4,5,ARMS AND AMMUNITION; PARTS AND ACCESSORIES THE...
...,...,...
93,94,"WADDING, FELT AND NONWOVENS; SPACIAL YARNS; TW..."
94,95,WOOD AND ARTICLES OF WOOD; WOOD CHARCOAL.
95,96,"WOOL, FINE OR COARSE ANIMAL HAIR, HORSEHAIR YA..."
96,97,WORKS OF ART COLLECTORS' PIECES AND ANTIQUES.


In [9]:
sql_statement = "SELECT * FROM Export"
df = pd.read_sql_query(sql_statement, conn_norm)
display(df)

,VALUEID,HSCODE,COUNTRY,YEAR,VALUE


In [10]:
# So the Export table was created but nothing has been added yet

In [11]:
# Grab the helper function for inserting entries
def insert_value(conn, values):
    sql = ''' INSERT INTO Export (HSCODE, COUNTRY, YEAR, VALUE)
              VALUES(?, ?, ?, ?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid

In [12]:
# Collect hscode and commodity entries
sql_statement = 'SELECT HSCODE, COMMODITY from COMMODITY'
values = execute_sql_statement(sql_statement, conn_norm)
print(values[:10])

[(1, 'AIRCRAFT, SPACECRAFT, AND PARTS THEREOF.'), (2, 'ALBUMINOIDAL SUBSTANCES; MODIFIED STARCHES; GLUES; ENZYMES.'), (3, 'ALUMINIUM AND ARTICLES THEREOF.'), (4, 'ANIMAL OR VEGETABLE FATS AND OILS AND THEIR CLEAVAGE PRODUCTS; PRE. EDIBLE FATS; ANIMAL OR VEGETABLE WAXEX.'), (5, 'ARMS AND AMMUNITION; PARTS AND ACCESSORIES THEREOF.'), (6, 'ARTICLES OF APPAREL AND CLOTHING ACCESSORIES, KNITTED OR CORCHETED.'), (7, 'ARTICLES OF APPAREL AND CLOTHING ACCESSORIES, NOT KNITTED OR CROCHETED.'), (8, 'ARTICLES OF IRON OR STEEL'), (9, 'ARTICLES OF LEATHER,SADDLERY AND HARNESS;TRAVEL GOODS, HANDBAGS AND SIMILAR CONT.ARTICLES OF ANIMAL GUT(OTHR THN SILK-WRM)GUT.'), (10, 'ARTICLES OF STONE, PLASTER, CEMENT, ASBESTOS, MICA OR SIMILAR MATERIALS.')]


In [13]:
# create a lookup dictionary that maps commodity 
# descriptions to hscode values
commodity_lookup = {}
for hscode, commodity in values:
    commodity_lookup[commodity] = hscode
    
print(commodity_lookup)

{'AIRCRAFT, SPACECRAFT, AND PARTS THEREOF.': 1, 'ALBUMINOIDAL SUBSTANCES; MODIFIED STARCHES; GLUES; ENZYMES.': 2, 'ALUMINIUM AND ARTICLES THEREOF.': 3, 'ANIMAL OR VEGETABLE FATS AND OILS AND THEIR CLEAVAGE PRODUCTS; PRE. EDIBLE FATS; ANIMAL OR VEGETABLE WAXEX.': 4, 'ARMS AND AMMUNITION; PARTS AND ACCESSORIES THEREOF.': 5, 'ARTICLES OF APPAREL AND CLOTHING ACCESSORIES, KNITTED OR CORCHETED.': 6, 'ARTICLES OF APPAREL AND CLOTHING ACCESSORIES, NOT KNITTED OR CROCHETED.': 7, 'ARTICLES OF IRON OR STEEL': 8, 'ARTICLES OF LEATHER,SADDLERY AND HARNESS;TRAVEL GOODS, HANDBAGS AND SIMILAR CONT.ARTICLES OF ANIMAL GUT(OTHR THN SILK-WRM)GUT.': 9, 'ARTICLES OF STONE, PLASTER, CEMENT, ASBESTOS, MICA OR SIMILAR MATERIALS.': 10, 'BEVERAGES, SPIRITS AND VINEGAR.': 11, 'CARPETS AND OTHER TEXTILE FLOOR COVERINGS.': 12, 'CERAMIC PRODUCTS.': 13, 'CEREALS.': 14, 'CLOCKS AND WATCHES AND PARTS THEREOF.': 15, 'COCOA AND COCOA PREPARATIONS.': 16, 'COFFEE, TEA, MATE AND SPICES.': 17, 'COPPER AND ARTICLES THEREOF.'

In [14]:
# Grab the data from the flat table and perform the 
# insertions into the normalized table
sql_statement = 'SELECT Commodity, country, year, value from export'
values = execute_sql_statement(sql_statement, conn)
# with conn_norm:
count = 0
with conn_norm:
    for value in values:
        text = value[0]
        hscode = commodity_lookup[text]
        insert_tuple = (hscode, value[1], value[2], value[3])
        insert_value(conn_norm, insert_tuple)
        

In [16]:
# This query will generate a table that matches the export
# table of the "flat" database
sql_statement = """
SELECT
export.hscode,
commodity.commodity,
export.country,
export.year,
export.value
FROM Export
JOIN Commodity ON Commodity.hscode = export.hscode
"""
df = pd.read_sql_query(sql_statement, conn_norm)
display(df)

,HSCODE,COMMODITY,COUNTRY,YEAR,VALUE
0,45,MEAT AND EDIBLE MEAT OFFAL.,AFGHANISTAN TIS,2018,0.18
1,28,"FISH AND CRUSTACEANS, MOLLUSCS AND OTHER AQUAT...",AFGHANISTAN TIS,2018,0.00
2,21,DAIRY PRODUCE; BIRDS' EGGS; NATURAL HONEY; EDI...,AFGHANISTAN TIS,2018,12.48
3,41,LIVE TREES AND OTHER PLANTS; BULBS; ROOTS AND ...,AFGHANISTAN TIS,2018,0.00
4,23,EDIBLE VEGETABLES AND CERTAIN ROOTS AND TUBERS.,AFGHANISTAN TIS,2018,1.89
...,...,...,...,...,...
133365,9,"ARTICLES OF LEATHER,SADDLERY AND HARNESS;TRAVE...",SAMOA,2010,0.00
133366,63,"PAPER AND PAPERBOARD; ARTICLES OF PAPER PULP, ...",SAMOA,2010,0.00
133367,71,"PRINTED BOOKDS, NEWSPAPERS, PICTURES AND OTHER...",SAMOA,2010,NaN
133368,20,COTTON.,SAMOA,2010,0.00


In [21]:
# we can keep track of the the time it takes for various
# operations in Python - use the time module
import time
# collect a time-stamp
time.monotonic()

2567341.578

In [22]:
conn_norm.close()

# time stamp prior to running some code
tic = time.monotonic()

# now run some code that we want to benchmark
conn_norm = create_connection('india_export_norm.db')
cur = conn_norm.cursor()
cur.execute('DROP TABLE EXPORT')
create_table_sql = """CREATE TABLE IF NOT EXISTS [Export] (
    [VALUEID] INTEGER NOT NULL PRIMARY KEY,
    [HSCODE] INTEGER NOT NULL, 
    [COUNTRY] TEXT NOT NULL,
    [YEAR] INTEGER NOT NULL,
    [VALUE] REAL,
    FOREIGN KEY(COUNTRY) REFERENCES Country(COUNTRY),
    FOREIGN KEY(YEAR) REFERENCES YEAR(YEAR),
    FOREIGN KEY(HSCODE) REFERENCES Commodity(HSCODE)
);
"""
create_table(conn_norm, create_table_sql)

with conn_norm:
    start_time = time.monotonic() 
    for value in values:
        text = value[0]
        hscode = commodity_lookup[text]
        insert_tuple = (hscode, value[1], value[2], value[3])
        insert_value(conn_norm, insert_tuple)
end_time = time.monotonic()  

# time stamp after running some code
toc = time.monotonic()

# the difference is the execution time
print(toc - tic)

4.891000000294298


In [23]:
# an laternative approach is to use a list of tuples
# and the executemany() function.
conn_norm.close()

# record the start time
tic = time.monotonic()

# alternative approach starts with assmbling a list
# of tuples to inssert
insert_tuples = []

for value in values:
    text = value[0]
    hscode = commodity_lookup[text]
    insert_tuple = (hscode, value[1], value[2], value[3])
    insert_tuples.append(insert_tuple)

conn_norm = create_connection('india_export_norm.db')
cur = conn_norm.cursor()
cur.execute('DROP TABLE EXPORT')
create_table_sql = """CREATE TABLE IF NOT EXISTS [Export] (
    [VALUEID] INTEGER NOT NULL PRIMARY KEY,
    [HSCODE] INTEGER NOT NULL, 
    [COUNTRY] TEXT NOT NULL,
    [YEAR] INTEGER NOT NULL,
    [VALUE] REAL,
    FOREIGN KEY(COUNTRY) REFERENCES Country(COUNTRY),
    FOREIGN KEY(YEAR) REFERENCES YEAR(YEAR),
    FOREIGN KEY(HSCODE) REFERENCES Commodity(HSCODE)
);
"""
create_table(conn_norm, create_table_sql)
sql = ''' INSERT INTO Export (HSCODE, COUNTRY, YEAR, VALUE)
              VALUES(?, ?, ?, ?) '''
cur = conn_norm.cursor()
cur.executemany(sql, insert_tuples)
conn_norm.commit()

# record the finish time
toc = time.monotonic()

print(toc - tic)


2.9529999997466803


In [24]:
# That's a good improvement
original_time = 4.891000000294298
alt_time = 2.9529999997466803
speedup = original_time / alt_time
speedup

1.6562817476172933

In [26]:
conn_norm.commit()

In [27]:
conn.close()
conn_norm.close()

In [29]:
# soccer database

# step 1 - use Pandas to read is as a dataframe
df = pd.read_csv('c:\\temp\\all_players.csv')
df

,Player,Club,POS,GP,GS,MINS,Year,Season
0,Roy Lassiter,TB,F,30,30,2580,1996,reg
1,Raul Diaz Arce,DC,F,28,28,2351,1996,reg
2,Eduardo Hurtado,LA,F,26,26,2323,1996,reg
3,Preki,KC,M,32,32,2880,1996,reg
4,Brian McBride,CLB,F,28,28,2307,1996,reg
...,...,...,...,...,...,...,...,...
15762,Alejandro Pozuelo,NaN,M,1,1,120,2020,post
15763,Franco Jara,NaN,F,2,2,210,2020,post
15764,Andy Polo,NaN,M,1,1,105,2020,post
15765,Cristian Roldan,NaN,M,4,4,360,2020,post


In [34]:
# isna() function returns True for n/a entries
# and Flase otherwise. Wrap the isna() result in a
# df[] clause to obtain a filtered list of players.
df[df['Club'].isna()]

,Player,Club,POS,GP,GS,MINS,Year,Season
9583,Raul Diaz Arce,NaN,F,6,6,544,1996,post
9584,Roy Lassiter,NaN,F,5,5,407,1996,post
9585,Preki,NaN,M,5,5,450,1996,post
9586,Eduardo Hurtado,NaN,F,6,5,499,1996,post
9587,Gerell Elliott,NaN,F,3,1,112,1996,post
...,...,...,...,...,...,...,...,...
15762,Alejandro Pozuelo,NaN,M,1,1,120,2020,post
15763,Franco Jara,NaN,F,2,2,210,2020,post
15764,Andy Polo,NaN,M,1,1,105,2020,post
15765,Cristian Roldan,NaN,M,4,4,360,2020,post


In [33]:
# Apply sum() opoeration to the isna() result to obtain a
# count of the unaffiliated players
df['Club'].isna().sum()

5493

In [40]:
# Let's list players who have more than 5 GP (games played)
df[df['GP'] == 33]

,Player,Club,POS,GP,GS,MINS,Year,Season
4570,Dwayne De Rosario,DC,M,33,31,2781,2011,reg
4620,Geoff Cameron,HOU,D,33,33,2970,2011,reg
4650,Osvaldo Alonso,MIN,M,33,32,2891,2011,reg
4785,Danny Califf,PHI,D,33,33,2970,2011,reg
4809,Gonzalo Segares,CHI,D,33,33,2950,2011,reg
...,...,...,...,...,...,...,...,...
9087,Ulises Segura,DC,M,33,19,1788,2019,reg
9108,Darlington Nagbe,CLB,M,33,32,2825,2019,reg
9115,Thomas McNamara,HOU,M,33,15,1530,2019,reg
9192,Mathieu Deplagne,CIN,D,33,33,2878,2019,reg


In [41]:
# create a "flat" (i.e. not normalized) sql dbase
conn = sqlite3.connect('all_players.db')
cur = conn.cursor()
df.to_sql('players', conn, if_exists='replace', index=False) # - writes the all_players.df to SQLIte DB
conn.commit()
conn.close()

In [42]:
# create some tables to make our database normalized
# club - distinct list of all clubs in the dbase
# position  - distinct list of all positions in the dbase
# year - distinct list of all years in the dbase
# season - distinct list of all seasons in the dbase
# player - list of all players with references to their club, position, etc.
# playerstats - list of player stastics per season

# first, create the normalized database
conn_norm = create_connection('players_norm.db', True)


In [44]:
# Create tables for club, position, year and season
conn = sqlite3.connect('all_players.db')
sql_statement = "SELECT DISTINCT CLUB FROM players"
rows = execute_sql_statement(sql_statement, conn)
rows = filter(lambda row: row[0], rows)
clubs = list(map(lambda row: row[0].strip(), rows))
clubs = list(set(clubs))

In [45]:
clubs

['CAN',
 'SLV',
 'CHI',
 'HOU',
 'LAFC',
 'ECU',
 'DAL',
 'HON',
 'POR',
 'ROC',
 'CLB',
 'NY',
 'NE',
 'CIN',
 'HAI',
 'MTL',
 'NSH',
 'NYR',
 'DC',
 'LA',
 'MET',
 'JAM',
 'PHI',
 'CIV',
 'TB',
 'MCF',
 'MIN',
 'RBNY',
 'SKC',
 'CHV',
 'MIA',
 'PAN',
 'MTQ',
 'GHA',
 'USA',
 'ATL',
 'RSL',
 'ORL',
 'COL',
 'TOR',
 'NYC',
 'VAN',
 'LFC',
 'KC',
 'SEA',
 'SJ']

In [46]:
# create the clubs table
create_table_sql = """CREATE TABLE IF NOT EXISTS[Clubs] (
    [Club] TEXT NOT NULL PRIMARY KEY
);
"""

create_table(conn_norm, create_table_sql) 

In [47]:
# define our insertion helper function
def insert_club(conn, values):
    sql = """INSERT INTO Clubs(Club)
                VALUES(?)"""
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid

# apply the insertion function to the unique list of clubs
with conn_norm:
    for club in clubs:
        insert_club(conn_norm, (club,))

In [48]:
sql_statement = "SELECT DISTINCT CLUB FROM Clubs"
rows = execute_sql_statement(sql_statement, conn_norm)
print(rows)

[('CAN',), ('SLV',), ('CHI',), ('HOU',), ('LAFC',), ('ECU',), ('DAL',), ('HON',), ('POR',), ('ROC',), ('CLB',), ('NY',), ('NE',), ('CIN',), ('HAI',), ('MTL',), ('NSH',), ('NYR',), ('DC',), ('LA',), ('MET',), ('JAM',), ('PHI',), ('CIV',), ('TB',), ('MCF',), ('MIN',), ('RBNY',), ('SKC',), ('CHV',), ('MIA',), ('PAN',), ('MTQ',), ('GHA',), ('USA',), ('ATL',), ('RSL',), ('ORL',), ('COL',), ('TOR',), ('NYC',), ('VAN',), ('LFC',), ('KC',), ('SEA',), ('SJ',)]


In [50]:
pd.read_sql("SELECT * FROM Clubs", conn_norm)

,Club
0,CAN
1,SLV
2,CHI
3,HOU
4,LAFC
5,ECU
6,DAL
7,HON
8,POR
9,ROC


In [51]:
# rinse and repeat for the positions table
sql_statement = "SELECT DISTINCT POS FROM players"
rows = execute_sql_statement(sql_statement, conn)
rows = filter(lambda row: row[0], rows)
positions = list(map(lambda row: row[0].strip(), rows))
positions = list(set(positions))

In [52]:
positions

['F', 'D', 'M', 'M-F', 'D-M', 'F-M', 'M-D']

In [53]:
create_table_sql = """CREATE TABLE IF NOT EXISTS[Positions] (
    [Position] TEXT NOT NULL PRIMARY KEY
);
"""

create_table(conn_norm, create_table_sql)

In [54]:
def insert_position(conn, values):
    sql = """INSERT INTO Positions(Position)
                VALUES(?)"""
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid


with conn_norm:
    for position in positions:
        insert_position(conn_norm, (position,))

In [55]:
pd.read_sql("SELECT * FROM Positions", conn_norm)

,Position
0,F
1,D
2,M
3,M-F
4,D-M
5,F-M
6,M-D


In [56]:
# repeat for the years table
sql_statement = "SELECT DISTINCT Year FROM players"
rows = execute_sql_statement(sql_statement, conn)
rows = filter(lambda row: row[0], rows)
years = list(map(lambda row: int(row[0]), rows))
years = list(set(years))


In [57]:
years

[1996,
 1997,
 1998,
 1999,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020]

In [58]:
create_table_sql = """CREATE TABLE IF NOT EXISTS[Years] (
    [Year] INTEGER NOT NULL PRIMARY KEY
);
"""

create_table(conn_norm, create_table_sql)

In [59]:
def insert_year(conn, values):
    sql = """INSERT INTO Years(Year)
                VALUES(?)"""
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid


with conn_norm:
    for year in years:
        insert_year(conn_norm, (year,))

In [60]:
pd.read_sql("SELECT * from YEARS", conn_norm)

,Year
0,1996
1,1997
2,1998
3,1999
4,2000
5,2001
6,2002
7,2003
8,2004
9,2005


In [61]:
# repeat again for the season
sql_statement = "SELECT DISTINCT season FROM players"
rows = execute_sql_statement(sql_statement, conn)
rows = filter(lambda row: row[0], rows)
seasons = list(map(lambda row: row[0].strip(), rows))
seasons = list(set(seasons))

In [62]:
seasons

['reg', 'post']

In [63]:
create_table_sql = """CREATE TABLE IF NOT EXISTS[Seasons] (
    [Season] TEXT NOT NULL PRIMARY KEY
);
"""

create_table(conn_norm, create_table_sql) 

In [64]:
def insert_season(conn, values):
    sql = """INSERT INTO Seasons(season)
                VALUES(?)"""
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid


with conn_norm:
    for season in seasons:
        insert_season(conn_norm, (season,))

In [65]:
pd.read_sql("SELECT * FROM SEASONS", conn_norm)

,Season
0,reg
1,post


In [66]:
ex_name=("Dwayne De Rosario",)

In [67]:
ex_name

('Dwayne De Rosario',)

In [68]:
ex_name = ex_name[0].strip()

In [69]:
ex_name

'Dwayne De Rosario'

In [70]:
name = ex_name.split(' ', 1)

In [71]:
name

['Dwayne', 'De Rosario']

In [72]:
len(name)

2

In [73]:
sql_statement = "SELECT DISTINCT player FROM players"
rows = execute_sql_statement(sql_statement, conn)
players = []
for row in rows:
    row = row[0].strip()
    name = row.split(' ', 1)
    if len(name) == 2:
        first, last = name
        first = first.strip()
        last = last.strip()
        players.append((first, last))
    elif len(name) == 1:
        first = name[0]
        players.append((first, None))

In [74]:
players

[('Roy', 'Lassiter'),
 ('Raul', 'Diaz Arce'),
 ('Eduardo', 'Hurtado'),
 ('Preki', None),
 ('Brian', 'McBride'),
 ('Steve', 'Rammel'),
 ('Vitalis', 'Takawira'),
 ('Paul', 'Bravo'),
 ('Jason', 'Kreis'),
 ('Giovanni', 'Savarese'),
 ('Pete', 'Marino'),
 ('Jean', 'Harbor'),
 ('Joe-Max', 'Moore'),
 ('Eric', 'Wynalda'),
 ('Eduardo', 'Espinoza'),
 ('Mark', 'Chung'),
 ('Shaun', 'Bartlett'),
 ('Giuseppe', 'Galderisi'),
 ('Cobi', 'Jones'),
 ('Steve', 'Ralston'),
 ('Marcelo', 'Balboa'),
 ('Adrian', 'Paz'),
 ('Alberto', 'Naveda'),
 ('Harut', 'Karapetyan'),
 ('Hugo', 'Sanchez'),
 ('Maurice', 'Johnston'),
 ('Antony', 'De Avila'),
 ('Mauricio', 'Cienfuegos'),
 ('Gerell', 'Elliott'),
 ('Martin', 'Vasquez'),
 ('Brian', 'Maisonneuve'),
 ('John', 'Kerr'),
 ('A.J.', 'Wood'),
 ('Steve', 'Trittschuh'),
 ('Diego', 'Viera'),
 ('Marco', 'Etcheverry'),
 ('Carlos', 'Valderrama'),
 ('Tony', 'Sanneh'),
 ('Mike', 'Sorber'),
 ('Darren', 'Sawatzky'),
 ('Miles', 'Joseph'),
 ('Brian', 'Haynes'),
 ('Paul', 'Keegan'),
 ('

In [75]:
create_table_sql = """CREATE TABLE IF NOT EXISTS[Players] (
    [PlayerID] INTEGER NOT NULL PRIMARY KEY,
    [FirstName] TEXT NOT NULL,
    [LastName] TEXT
);
"""

create_table(conn_norm, create_table_sql)

In [76]:
def insert_player(conn, values):
    sql = """INSERT INTO Players(FirstName, LastName)
                VALUES(?, ?)"""
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid


with conn_norm:
    for player in players:
        insert_player(conn_norm, player)

In [77]:
pd.read_sql("SELECT * From Players", conn_norm)

,PlayerID,FirstName,LastName
0,1,Roy,Lassiter
1,2,Raul,Diaz Arce
2,3,Eduardo,Hurtado
3,4,Preki,None
4,5,Brian,McBride
...,...,...,...
3299,3300,Andres,Reyes
3300,3301,Dylan,Nealis
3301,3302,Luis,Binks
3302,3303,Siem,de Jong


In [78]:
# now create a playerstats table that leverages the other
# tables
create_table_sql = """CREATE TABLE IF NOT EXISTS[PlayerStats] (
    [StatID] INTEGER NOT NULL PRIMARY KEY,
    [PlayerID] INTEGER NOT NULL,
    [Club] TEXT,
    [Position] TEXT NOT NULL,
    [GP] INTEGER NOT NULL,
    [GS] INTEGER NOT NULL,
    [MIN] INTEGER NOT NULL,
    [Year] INTEGER NOT NULL,
    [Season] TEXT NOT NULL,
    FOREIGN KEY(PlayerID) REFERENCES Players(PlayerID),
    FOREIGN KEY(Club) REFERENCES Clubs(Club),
    FOREIGN KEY(Year) REFERENCES Years(Year),
    FOREIGN KEY(Season) REFERENCES Seasons(Season)     
);
"""

create_table(conn_norm, create_table_sql)

In [79]:
# create lookup for player name to player id
sql_statement = "SELECT * FROM players"
rows = execute_sql_statement(sql_statement, conn_norm)
player_lookup_dict = {}
for row in rows:
    pid, first, last = row
    if last:
        name = f'{first} {last}'
        player_lookup_dict[name] = pid
    else:
        player_lookup_dict[first] = pid

In [80]:
player_lookup_dict

{'Roy Lassiter': 1,
 'Raul Diaz Arce': 2,
 'Eduardo Hurtado': 3,
 'Preki': 4,
 'Brian McBride': 5,
 'Steve Rammel': 6,
 'Vitalis Takawira': 7,
 'Paul Bravo': 8,
 'Jason Kreis': 9,
 'Giovanni Savarese': 10,
 'Pete Marino': 11,
 'Jean Harbor': 12,
 'Joe-Max Moore': 13,
 'Eric Wynalda': 14,
 'Eduardo Espinoza': 15,
 'Mark Chung': 16,
 'Shaun Bartlett': 17,
 'Giuseppe Galderisi': 18,
 'Cobi Jones': 19,
 'Steve Ralston': 20,
 'Marcelo Balboa': 21,
 'Adrian Paz': 22,
 'Alberto Naveda': 23,
 'Harut Karapetyan': 24,
 'Hugo Sanchez': 25,
 'Maurice Johnston': 26,
 'Antony De Avila': 27,
 'Mauricio Cienfuegos': 28,
 'Gerell Elliott': 29,
 'Martin Vasquez': 30,
 'Brian Maisonneuve': 31,
 'John Kerr': 32,
 'A.J. Wood': 33,
 'Steve Trittschuh': 34,
 'Diego Viera': 35,
 'Marco Etcheverry': 36,
 'Carlos Valderrama': 37,
 'Tony Sanneh': 38,
 'Mike Sorber': 39,
 'Darren Sawatzky': 40,
 'Miles Joseph': 41,
 'Brian Haynes': 42,
 'Paul Keegan': 43,
 'Rob Johnson': 44,
 'Marcelo Carrera': 45,
 'Washington R

In [81]:
conn.close()
conn_norm.close()